In [ ]:
!gdown --id 1_Zds0QWmbn7bYn3iGrS1gyTLv33ZmwzU

In [ ]:
!unzip "/content/Memes-final.zip"
!rm Memes-final.zip

In [ ]:
!mv Memes\ \(final\) Memes-final

In [ ]:
!pip install evaluation

In [ ]:
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
import numpy as np
import keras
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import ResNet152, EfficientNetV2L
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


import os
import gc
import shutil 

In [ ]:
#path
# csv_path_test = '/content/MAMI/TEST/Test.csv'
# csv_path_train = '/content/MAMI/TRAINING/training.csv'
image_path = '/content/Memes-final/'

In [ ]:
batch_size = 32
epochs = 50
image_size = 224

In [ ]:
def loadImage(image_path): # /content/MAMI/TRAINING
    try:
        return load_img(image_path, target_size=(image_size, image_size))
    except:
        image_path = image_path.replace('png', 'jpg')
        return load_img(image_path, target_size=(image_size, image_size))

In [ ]:
train_df = pd.read_excel('/content/Memes-final/final.xlsx')
train_df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'], inplace=True)
# train_df.dropna(inplace=True)

path = image_path+'/'

train_df['image_path'] = path + train_df['file_name']

In [ ]:
train_df

In [ ]:
train_df['image'] = None
train_df['image'] = train_df['image_path'].apply(lambda x: img_to_array(loadImage(x))) 

In [ ]:
train_df, test_df = train_test_split(train_df, test_size=0.1, random_state=41)

In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [ ]:
print(len(train_df))
len(test_df)

746


93

In [ ]:
X_train = train_df[['file_name', 'image']]
y_train = train_df['hate_type2']

X_test = test_df[['file_name', 'image']]
y_test = test_df['hate_type2']

In [ ]:
tmp = []
for value in X_train['image']:
  tmp.append(value) 
iX_train = np.array(tmp)

In [ ]:
y_train = to_categorical(y_train, 5)

In [ ]:
tmp = []
for value in X_test['image']:
  tmp.append(value) 
iX_test = np.array(tmp)

In [ ]:
y_test= to_categorical(y_test, 5)

In [ ]:
# tmp = []
# for value in y_train:
#     tmp.append([value])  
# y_train = np.array(tmp)

In [ ]:
model_path = '/content/Best_model'

# earlystopping_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    monitor='val_accuracy',
    mode='max',
    save_weights_only=True,
    save_best_only=True,
    verbose=1)

In [ ]:
#_______________________________IMAGE MODEL_______________________________
l2_strength = 1e-5

input_image = layers.Input(shape=(image_size,image_size,3))
model = EfficientNetV2L(input_tensor=input_image, weights='imagenet', include_top=False)

for layer in model.layers:
    layer.trainable = False

x = model.output
x = layers.Flatten(input_shape=model.output_shape[1:])(x)
x = layers.Dense(350, activation='relu', kernel_regularizer=regularizers.l2(l2_strength))(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_strength))(x)
x = layers.Dense(5, activation='softmax')(x)

image_model = Model(model.input, x)

image_model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
# image_model.summary()

history = image_model.fit(iX_train, 
                    y_train,
                    validation_data=(iX_test, y_test), # Remove
                    epochs= epochs,
                    batch_size=batch_size,
                    callbacks=[checkpoint_callback]
)   

In [ ]:
model_output = image_model.predict(iX_test) 

In [ ]:
preds = []
for out in model_output:
  preds.append(np.argmax(out))

In [ ]:
labels = []
for x in y_test:
  labels.append(np.argmax(x))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print("F1 macro:   {}".format(round(f1_score(labels, preds, average="macro"), 3)))
print("Precision:   {}".format(precision_score(labels,preds, average="macro"), 3))
print("Recall:   {}".format(recall_score(labels,preds, average="macro"), 3))
print("F1 micro:   {}".format(round(f1_score(labels, preds, average="micro"), 3)))